# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

## Initializing Spark and Loading Data

In [1]:
import findspark
findspark.init('/home/shahayush954/spark-3.4.1-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lgr_exercise').getOrCreate()

23/08/19 14:32:24 WARN Utils: Your hostname, ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/19 14:32:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 14:32:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [5]:
relevant_data = data.select(['Age','Total_Purchase','Years','Num_Sites','Churn'])

In [9]:
relevant_data.show()

+----+--------------+-----+---------+-----+
| Age|Total_Purchase|Years|Num_Sites|Churn|
+----+--------------+-----+---------+-----+
|42.0|       11066.8| 7.22|      8.0|    1|
|41.0|      11916.22|  6.5|     11.0|    1|
|38.0|      12884.75| 6.67|     12.0|    1|
|42.0|       8010.76| 6.71|     10.0|    1|
|37.0|       9191.58| 5.56|      9.0|    1|
|48.0|      10356.02| 5.12|      8.0|    1|
|44.0|      11331.58| 5.23|     11.0|    1|
|32.0|       9885.12| 6.92|      9.0|    1|
|43.0|       14062.6| 5.46|     11.0|    1|
|40.0|       8066.94| 7.11|     11.0|    1|
|30.0|      11575.37| 5.22|      8.0|    1|
|45.0|       8771.02| 6.64|     11.0|    1|
|45.0|       8988.67| 4.84|     11.0|    1|
|40.0|       8283.32|  5.1|     13.0|    1|
|41.0|       6569.87|  4.3|     11.0|    1|
|38.0|      10494.82| 6.81|     12.0|    1|
|45.0|       8213.41| 7.35|     11.0|    1|
|43.0|      11226.88| 8.08|     12.0|    1|
|53.0|       5515.09| 6.85|      8.0|    1|
|46.0|        8046.4| 5.69|     

In [6]:
data_without_null = relevant_data.na.drop()

In [11]:
data_without_null.show()

+----+--------------+-----+---------+-----+
| Age|Total_Purchase|Years|Num_Sites|Churn|
+----+--------------+-----+---------+-----+
|42.0|       11066.8| 7.22|      8.0|    1|
|41.0|      11916.22|  6.5|     11.0|    1|
|38.0|      12884.75| 6.67|     12.0|    1|
|42.0|       8010.76| 6.71|     10.0|    1|
|37.0|       9191.58| 5.56|      9.0|    1|
|48.0|      10356.02| 5.12|      8.0|    1|
|44.0|      11331.58| 5.23|     11.0|    1|
|32.0|       9885.12| 6.92|      9.0|    1|
|43.0|       14062.6| 5.46|     11.0|    1|
|40.0|       8066.94| 7.11|     11.0|    1|
|30.0|      11575.37| 5.22|      8.0|    1|
|45.0|       8771.02| 6.64|     11.0|    1|
|45.0|       8988.67| 4.84|     11.0|    1|
|40.0|       8283.32|  5.1|     13.0|    1|
|41.0|       6569.87|  4.3|     11.0|    1|
|38.0|      10494.82| 6.81|     12.0|    1|
|45.0|       8213.41| 7.35|     11.0|    1|
|43.0|      11226.88| 8.08|     12.0|    1|
|53.0|       5515.09| 6.85|      8.0|    1|
|46.0|        8046.4| 5.69|     

## Logistic Regression

In [7]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Years','Num_Sites'], outputCol='features')

In [8]:
from pyspark.ml.classification import LogisticRegression
lr_model = LogisticRegression(featuresCol='features', labelCol='Churn')

In [9]:
from pyspark.ml.pipeline import Pipeline
pipeline = Pipeline(stages=[assembler, lr_model])

In [10]:
train_data, test_data = data_without_null.randomSplit([0.7, 0.3])

In [11]:
model = pipeline.fit(train_data)

In [12]:
results = model.transform(test_data)

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [14]:
my_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [15]:
results.select(['prediction', 'Churn']).show()

+----------+-----+
|prediction|Churn|
+----------+-----+
|       0.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       1.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
+----------+-----+
only showing top 20 rows



In [24]:
results.head(50)

[Row(Age=22.0, Total_Purchase=11254.38, Years=4.96, Num_Sites=8.0, Churn=0, features=DenseVector([22.0, 11254.38, 4.96, 8.0]), rawPrediction=DenseVector([4.7716, -4.7716]), probability=DenseVector([0.9916, 0.0084]), prediction=0.0),
 Row(Age=28.0, Total_Purchase=8670.98, Years=3.99, Num_Sites=6.0, Churn=0, features=DenseVector([28.0, 8670.98, 3.99, 6.0]), rawPrediction=DenseVector([7.4008, -7.4008]), probability=DenseVector([0.9994, 0.0006]), prediction=0.0),
 Row(Age=28.0, Total_Purchase=11204.23, Years=3.67, Num_Sites=11.0, Churn=0, features=DenseVector([28.0, 11204.23, 3.67, 11.0]), rawPrediction=DenseVector([1.7893, -1.7893]), probability=DenseVector([0.8568, 0.1432]), prediction=0.0),
 Row(Age=28.0, Total_Purchase=11245.38, Years=6.72, Num_Sites=8.0, Churn=0, features=DenseVector([28.0, 11245.38, 6.72, 8.0]), rawPrediction=DenseVector([3.3331, -3.3331]), probability=DenseVector([0.9655, 0.0345]), prediction=0.0),
 Row(Age=30.0, Total_Purchase=7960.64, Years=2.72, Num_Sites=10.0, C

In [16]:
AUC = my_evaluator.evaluate(results)

In [17]:
AUC

0.7317351598173516

In [18]:
new_data = spark.read.csv('new_customers.csv', header=True, inferSchema=True)

In [19]:
new_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [20]:
relevant_new_data = new_data.select(['Age','Total_Purchase','Years','Num_Sites'])

In [21]:
new_data_without_null = relevant_new_data.na.drop()

In [23]:
new_data_without_null.show()

+----+--------------+-----+---------+
| Age|Total_Purchase|Years|Num_Sites|
+----+--------------+-----+---------+
|37.0|       9935.53| 7.71|      8.0|
|23.0|       7526.94| 9.28|     15.0|
|65.0|         100.0|  1.0|     15.0|
|32.0|        6487.5|  9.4|     14.0|
|32.0|      13147.71| 10.0|      8.0|
|22.0|       8445.26| 3.46|     14.0|
+----+--------------+-----+---------+



In [24]:
new_results = model.transform(new_data_without_null)

In [25]:
new_results.show()

+----+--------------+-----+---------+--------------------+--------------------+--------------------+----------+
| Age|Total_Purchase|Years|Num_Sites|            features|       rawPrediction|         probability|prediction|
+----+--------------+-----+---------+--------------------+--------------------+--------------------+----------+
|37.0|       9935.53| 7.71|      8.0|[37.0,9935.53,7.7...|[2.41042080493816...|[0.91761849812381...|       0.0|
|23.0|       7526.94| 9.28|     15.0|[23.0,7526.94,9.2...|[-6.2012561873088...|[0.00202278294769...|       1.0|
|65.0|         100.0|  1.0|     15.0|[65.0,100.0,1.0,1...|[-3.4419470089084...|[0.03100992637852...|       1.0|
|32.0|        6487.5|  9.4|     14.0|[32.0,6487.5,9.4,...|[-5.6391724970495...|[0.00354321064952...|       1.0|
|32.0|      13147.71| 10.0|      8.0|[32.0,13147.71,10...|[1.21993210268943...|[0.77205160054059...|       0.0|
|22.0|       8445.26| 3.46|     14.0|[22.0,8445.26,3.4...|[-1.1187200286093...|[0.24624878227916...|    

In [27]:
new_results.withColumn('Account Manager', new_results['prediction']).show()

+----+--------------+-----+---------+--------------------+--------------------+--------------------+----------+---------------+
| Age|Total_Purchase|Years|Num_Sites|            features|       rawPrediction|         probability|prediction|Account Manager|
+----+--------------+-----+---------+--------------------+--------------------+--------------------+----------+---------------+
|37.0|       9935.53| 7.71|      8.0|[37.0,9935.53,7.7...|[2.41042080493816...|[0.91761849812381...|       0.0|            0.0|
|23.0|       7526.94| 9.28|     15.0|[23.0,7526.94,9.2...|[-6.2012561873088...|[0.00202278294769...|       1.0|            1.0|
|65.0|         100.0|  1.0|     15.0|[65.0,100.0,1.0,1...|[-3.4419470089084...|[0.03100992637852...|       1.0|            1.0|
|32.0|        6487.5|  9.4|     14.0|[32.0,6487.5,9.4,...|[-5.6391724970495...|[0.00354321064952...|       1.0|            1.0|
|32.0|      13147.71| 10.0|      8.0|[32.0,13147.71,10...|[1.21993210268943...|[0.77205160054059...|    